In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# ============================================
# Cell 1: 環境設定と初期化
# ============================================

import os
import sys
import json
import pickle
import pandas as pd
import numpy as np
import time
from pathlib import Path
from typing import Dict, Any, List, Optional

# パスの設定
BASE_DIR = Path("/home/asomura/nextstep")
sys.path.insert(0, str(BASE_DIR))

# _compatを使用（オプション）
try:
    from _compat import paths
    cfg = paths.load_config(os.getenv("CONFIG_JSON"), strict=False)
    RUN_ID = cfg.get("run_id") or os.getenv("RUN_ID") or "20251030_004055"
except ImportError:
    RUN_ID = "20251030_004055"
    print("[INFO] _compat not found, using default RUN_ID")

ARTIFACTS_DIR = BASE_DIR / "artifacts" / RUN_ID
HANDOFF_DIR = ARTIFACTS_DIR / "handoff"
LOGS_DIR = ARTIFACTS_DIR / "logs"

print(f"[INFO] BASE_DIR = {BASE_DIR}")
print(f"[INFO] RUN_ID = {RUN_ID}")
print(f"[INFO] ARTIFACTS_DIR = {ARTIFACTS_DIR}")

# 出力用のタイムスタンプ
import datetime
timestamp = datetime.datetime.now().strftime("%Y-%m-%d_%H%M%S")
print(f"[INFO] Timestamp: {timestamp}")

[INFO] BASE_DIR = /home/asomura/nextstep
[INFO] RUN_ID = 2025-11-22_020618
[INFO] ARTIFACTS_DIR = /home/asomura/nextstep/artifacts/2025-11-22_020618
[INFO] Timestamp: 2025-11-22_020618


In [3]:
# ============================================
# Cell 2: External Dataの読み込みとBrand Keywords補強
# ============================================

# handoffからexternal_dataを読み込み
handoff_path = HANDOFF_DIR / "04-3_llm_tools_setup_with_tools.pkl"

if not handoff_path.exists():
    print(f"[ERROR] Handoff file not found: {handoff_path}")
    print("[INFO] Trying alternative path...")
    handoff_path = Path("/home/asomura/nextstep/artifacts/20251030_004055/handoff/04-3_llm_tools_setup_with_tools.pkl")

with open(handoff_path, 'rb') as f:
    external_data = pickle.load(f)

print(f"[INFO] external_data loaded from: {handoff_path}")
print(f"[INFO] external_data keys: {list(external_data.keys())[:8]} ...")

# brand_keywordsの確認と補強
if 'brand_keywords' not in external_data:
    external_data['brand_keywords'] = []
    print("[WARNING] brand_keywords not found, creating new list")

# 現在のブランドを小文字で取得
brands_lower = [b.lower() for b in external_data['brand_keywords'] if isinstance(b, str)]
print(f"[INFO] Current brand_keywords count: {len(external_data['brand_keywords'])}")

# 重要ブランドのリスト（日本の金融機関を重点的に）
essential_brands = {
    # 日本の金融機関
    "mufg", "smbc", "mizuho", "ufj", "mitsubishi", "sumitomo", "mitsui",
    "resona", "jcb", "visa", "mastercard", "amex",
    "japanpost", "jppost", "yucho", "jabank", "shinkin",
    # 日本のサービス
    "rakuten", "mercari", "yahoo", "line", "docomo", "au", "softbank",
    "ntt", "kddi", "uniqlo", "muji", "nitori",
    # グローバルブランド
    "amazon", "google", "apple", "microsoft", "paypal", "netflix",
    "facebook", "meta", "instagram", "twitter", "x",
    "metamask", "binance", "coinbase", "ethereum", "bitcoin",
    # 短縮URL・CDN
    "bit", "tinyurl", "shorturl", "cdn", "cloudflare", "akamai"
}

# 不足しているブランドを追加
added_brands = []
for brand in essential_brands:
    if brand not in brands_lower:
        external_data['brand_keywords'].append(brand)
        added_brands.append(brand)

print(f"[INFO] Added {len(added_brands)} new brands")
print(f"[INFO] Total brand_keywords: {len(external_data['brand_keywords'])}")

# 主要ブランドの確認表示
check_brands = ["mufg", "smbc", "amazon", "mercari", "rakuten", "metamask"]
print("\n[Brand Check]")
for brand in check_brands:
    exists = brand in [b.lower() for b in external_data['brand_keywords']]
    status = "✅" if exists else "❌"
    print(f"  {brand:15} {status}")

[ERROR] Handoff file not found: /home/asomura/nextstep/artifacts/2025-11-22_020618/handoff/04-3_llm_tools_setup_with_tools.pkl
[INFO] Trying alternative path...
[INFO] external_data loaded from: /home/asomura/nextstep/artifacts/20251030_004055/handoff/04-3_llm_tools_setup_with_tools.pkl
[INFO] external_data keys: ['cfg', 'brand_keywords', 'cert_full_info_map', 'tools', 'llm', 'async_client', 'tools_code', 'present_top'] ...
[INFO] Current brand_keywords count: 100
[INFO] Added 39 new brands
[INFO] Total brand_keywords: 139

[Brand Check]
  mufg            ✅
  smbc            ✅
  amazon          ✅
  mercari         ✅
  rakuten         ✅
  metamask        ✅


In [4]:
# ============================================
# Cell 2の最後に追加: 04-2から不足データを補完
# ============================================

print("\n" + "=" * 80)
print("[INFO] Loading additional data from 04-2")
print("=" * 80)

# 04-2のパスを探索
pickle_04_2_paths = [
    HANDOFF_DIR / "04-2_statistical_analysis.pkl",
    Path("/home/asomura/nextstep/artifacts/20251030_004055/handoff/04-2_statistical_analysis.pkl"),
]

data_04_2 = None
for path in pickle_04_2_paths:
    if path.exists():
        print(f"[INFO] Found 04-2 at: {path}")
        try:
            with open(path, 'rb') as f:
                data_04_2 = pickle.load(f)
            print(f"[INFO] 04-2 loaded successfully")
            break
        except Exception as e:
            print(f"[WARN] Failed to load {path}: {e}")

if data_04_2 is None:
    print("[WARN] 04-2 not found, continuing with available data")
else:
    print(f"[INFO] 04-2 keys: {list(data_04_2.keys())[:10]}...")
    
    # TLD関連データを追加
    tld_keys = [
        'dangerous_tlds', 'DANGEROUS_TLDS',
        'legitimate_tlds', 'LEGITIMATE_TLDS',
        'neutral_tlds', 'NEUTRAL_TLDS',
        'phishing_tld_stats', 'TLD_STATS', 'tld_stats'
    ]
    
    added_count = 0
    for key in tld_keys:
        if key in data_04_2:
            # 標準化されたキー名にマッピング
            standard_key = key.lower()
            if 'dangerous' in standard_key:
                target_key = 'dangerous_tlds'
            elif 'legitimate' in standard_key:
                target_key = 'legitimate_tlds'
            elif 'neutral' in standard_key:
                target_key = 'neutral_tlds'
            elif 'tld' in standard_key and 'stat' in standard_key:
                target_key = 'phishing_tld_stats'
            else:
                target_key = key
            
            if target_key not in external_data:
                external_data[target_key] = data_04_2[key]
                value = data_04_2[key]
                size = len(value) if hasattr(value, '__len__') else 'N/A'
                print(f"  ✅ Added {target_key}: {type(value).__name__} (len={size})")
                added_count += 1
    
    # High Risk Words を追加
    hrw_keys = ['HIGH_RISK_WORDS', 'high_risk_words', 'high_risk']
    for key in hrw_keys:
        if key in data_04_2 and 'high_risk_words' not in external_data:
            external_data['high_risk_words'] = data_04_2[key]
            words = data_04_2[key]
            print(f"  ✅ Added high_risk_words: {type(words).__name__} (len={len(words)})")
            added_count += 1
            break
    
    # Known Domains を追加
    kd_keys = ['KNOWN_DOMAINS', 'known_domains', 'legitimate_domains']
    for key in kd_keys:
        if key in data_04_2 and 'known_domains' not in external_data:
            external_data['known_domains'] = data_04_2[key]
            domains = data_04_2[key]
            size = len(domains) if hasattr(domains, '__len__') else 'N/A'
            print(f"  ✅ Added known_domains: {type(domains).__name__} (len={size})")
            added_count += 1
            break
    
    print(f"\n[INFO] Added {added_count} data items from 04-2")

# 最終確認
print("\n" + "=" * 80)
print("[INFO] Final Data Availability Check")
print("=" * 80)

required_keys = {
    'brand_keywords': 'Brand keywords list',
    'cert_full_info_map': 'Certificate information',
    'dangerous_tlds': 'Dangerous TLDs',
    'legitimate_tlds': 'Legitimate TLDs',
    'neutral_tlds': 'Neutral TLDs',
    'phishing_tld_stats': 'TLD phishing statistics',
    'high_risk_words': 'High risk words',
    'known_domains': 'Known legitimate domains'
}

for key, description in required_keys.items():
    if key in external_data:
        value = external_data[key]
        size = len(value) if hasattr(value, '__len__') else 'N/A'
        print(f"  ✅ {description:30} (len={size})")
    else:
        print(f"  ❌ {description:30} NOT AVAILABLE")


[INFO] Loading additional data from 04-2
[INFO] Found 04-2 at: /home/asomura/nextstep/artifacts/20251030_004055/handoff/04-2_statistical_analysis.pkl
[INFO] 04-2 loaded successfully
[INFO] 04-2 keys: ['cfg', 'RUN_ID', 'SESSION_ID', 'output_dirs', 'brand_keywords', 'cert_full_info_map', 'false_negatives_df', 'fn_features_df', 'HIGH_RISK_WORDS', 'suspicious_words_stats']...
  ✅ Added dangerous_tlds: NoneType (len=N/A)
  ✅ Added legitimate_tlds: NoneType (len=N/A)
  ✅ Added neutral_tlds: NoneType (len=N/A)
  ✅ Added phishing_tld_stats: DataFrame (len=178)
  ✅ Added high_risk_words: list (len=100)
  ✅ Added known_domains: dict (len=4132)

[INFO] Added 6 data items from 04-2

[INFO] Final Data Availability Check
  ✅ Brand keywords list            (len=139)
  ✅ Certificate information        (len=4132)
  ✅ Dangerous TLDs                 (len=N/A)
  ✅ Legitimate TLDs                (len=N/A)
  ✅ Neutral TLDs                   (len=N/A)
  ✅ TLD phishing statistics        (len=178)
  ✅ High ri

In [5]:
# ============================================
# Cell 3: Random100データの準備（元のノートブック準拠）
# ============================================

from urllib.parse import urlparse

# データ探索用ヘルパー関数
def _search_eval_df(obj):
    '''dict/list 再帰で DataFrame を探す'''
    if obj is None:
        return None
    if isinstance(obj, pd.DataFrame):
        return obj
    if isinstance(obj, dict):
        # よくあるキー直参照
        for k in ("false_negatives_df", "fn_df", "eval_df", "random_eval_df"):
            v = obj.get(k)
            if isinstance(v, pd.DataFrame):
                return v
        for v in obj.values():
            hit = _search_eval_df(v)
            if hit is not None:
                return hit
    if isinstance(obj, (list, tuple)):
        for v in obj:
            hit = _search_eval_df(v)
            if hit is not None:
                return hit
    return None

def _normalize_eval_df(df):
    '''列名の揺れに対応して domain, ml_probability の2列に正規化'''
    print("[DEBUG] columns:", list(df.columns))
    
    # lower → 元名のマップ
    lower2orig = {c.lower(): c for c in df.columns}
    
    # 1) domain候補
    domain_candidates = ["domain", "fqdn", "domain_name", "hostname", "host", "requested_host"]
    domain_key = None
    for key in domain_candidates:
        if key in lower2orig:
            domain_key = lower2orig[key]
            break
    
    if domain_key is None:
        # 部分一致
        for c in df.columns:
            if any(kw in c.lower() for kw in ["domain", "fqdn", "host", "url"]):
                domain_key = c
                print(f"[DEBUG] domain fallback: {c}")
                break
    
    # 2) ml_probability候補
    ml_candidates = ["ml_probability", "ml_prob", "probability", "prediction_proba", 
                     "score", "pred_proba", "proba", "confidence"]
    mlp_key = None
    for key in ml_candidates:
        if key in lower2orig:
            mlp_key = lower2orig[key]
            break
    
    if mlp_key is None:
        # floatカラムから推測
        float_cols = [c for c in df.columns if pd.api.types.is_float_dtype(df[c])]
        for c in float_cols:
            if any(kw in c.lower() for kw in ["prob", "score", "pred"]):
                mlp_key = c
                print(f"[DEBUG] ml_probability fallback: {c}")
                break
    
    if domain_key is None or mlp_key is None:
        print("[ERROR] could not infer domain/ml_probability columns")
        return None
    
    print(f"[DEBUG] chosen columns: domain={domain_key}, ml_prob={mlp_key}")
    
    tmp = df[[domain_key, mlp_key]].copy()
    tmp.columns = ["domain", "ml_probability"]
    
    # url→domainの正規化
    if "url" in domain_key.lower():
        def _to_domain(x):
            if not isinstance(x, str):
                return ""
            if "://" in x:
                netloc = urlparse(x).netloc
            else:
                netloc = x
            netloc = netloc.split("@")[-1].split(":")[0]
            return netloc.lower()
        tmp["domain"] = tmp["domain"].map(_to_domain)
    
    # クリーンアップ
    tmp["domain"] = tmp["domain"].astype(str).str.strip().str.lower()
    tmp = tmp[tmp["domain"].str.len() > 0]
    tmp["ml_probability"] = pd.to_numeric(tmp["ml_probability"], errors="coerce")
    tmp = tmp[(tmp["ml_probability"] >= 0.0) & (tmp["ml_probability"] <= 1.0)]
    tmp = tmp.dropna(subset=["ml_probability"]).reset_index(drop=True)
    
    return tmp

# Random100データを取得
def get_random100_domains():
    '''Random100のドメインリストを取得（元のノートブックのロジック準拠）'''
    
    # 方法1: 既存のCSVから
    csv_paths = [
        LOGS_DIR / "random_eval_domains_latest.csv",
        ARTIFACTS_DIR / "logs" / "random_eval_domains_latest.csv",
        Path("/home/asomura/nextstep/artifacts/2025-11-16_184237/logs/random_eval_domains_latest.csv")
    ]
    
    for csv_path in csv_paths:
        if csv_path.exists():
            print(f"[INFO] Loading existing CSV from: {csv_path}")
            df = pd.read_csv(csv_path)
            return df
    
    # 方法2: pickleから生成（元のノートブックのロジック）
    print("[INFO] CSV not found, generating from pickle...")
    
    pickle_paths = [
        HANDOFF_DIR / "04-2_statistical_analysis.pkl",
        HANDOFF_DIR / "04-3_llm_tools_setup_with_tools.pkl",
        Path("/home/asomura/nextstep/artifacts/20251030_004055/handoff/04-2_statistical_analysis.pkl"),
        Path("/home/asomura/nextstep/artifacts/20251030_004055/handoff/04-3_llm_tools_setup_with_tools.pkl")
    ]
    
    eval_source_df = None
    for pickle_path in pickle_paths:
        if not pickle_path.exists():
            continue
            
        print(f"[INFO] Trying: {pickle_path}")
        try:
            with open(pickle_path, 'rb') as f:
                obj = pickle.load(f)
            
            # DataFrameを探す
            raw_df = _search_eval_df(obj)
            if raw_df is not None and len(raw_df) > 0:
                print(f"[INFO] Found DataFrame with {len(raw_df)} rows")
                eval_source_df = raw_df
                break
        except Exception as e:
            print(f"[WARN] Failed to load {pickle_path}: {e}")
    
    if eval_source_df is None:
        raise RuntimeError(
            "評価用DataFrameが見つかりません。\n"
            "04-2 または 04-3 のpickleファイルを確認してください。"
        )
    
    # 正規化
    norm_df = _normalize_eval_df(eval_source_df)
    if norm_df is None:
        raise RuntimeError("DataFrameの正規化に失敗しました")
    
    print(f"[INFO] Source rows (normalized): {len(norm_df)}")
    
    # ランダムサンプリング（元のノートブックは固定シードなし）
    sample_n = min(100, len(norm_df))
    random100 = norm_df.sample(n=sample_n).reset_index(drop=True)
    
    # CSV保存
    out_csv = LOGS_DIR / "random_eval_domains_latest.csv"
    LOGS_DIR.mkdir(parents=True, exist_ok=True)
    random100.to_csv(out_csv, index=False, encoding="utf-8")
    print(f"[INFO] Random100 saved to: {out_csv}")
    
    return random100

# Random100データを取得
random100_df = get_random100_domains()
print(f"[INFO] Random100 loaded: {len(random100_df)} domains")
print(random100_df.head())

# ML確率の分布を確認
print(f"\n[ML Probability Distribution]")
print(f"  Min:    {random100_df['ml_probability'].min():.3f}")
print(f"  Max:    {random100_df['ml_probability'].max():.3f}")
print(f"  Mean:   {random100_df['ml_probability'].mean():.3f}")
print(f"  Median: {random100_df['ml_probability'].median():.3f}")

# 高リスクドメイン（ML確率 > 0.4）の数
high_risk = random100_df[random100_df['ml_probability'] > 0.4]
print(f"  High risk (>0.4): {len(high_risk)} domains")

[INFO] Loading existing CSV from: /home/asomura/nextstep/artifacts/2025-11-16_184237/logs/random_eval_domains_latest.csv
[INFO] Random100 loaded: 100 domains
            domain  ml_probability
0          jmbf.cn        0.453648
1  baidu-xiamen.cn        0.244429
2      tsjianye.cn        0.236587
3    internetku.id        0.129556
4  teslafarmer.com        0.094156

[ML Probability Distribution]
  Min:    0.004
  Max:    0.497
  Mean:   0.201
  Median: 0.179
  High risk (>0.4): 10 domains


In [6]:
# ============================================
# Cell 4: LangGraphエージェントの初期化（Random100用完全版）
# ============================================

import sys
import os
import types

# 環境設定
os.chdir(BASE_DIR)
phishing_agent_path = BASE_DIR / "phishing_agent"
if str(phishing_agent_path) not in sys.path:
    sys.path.insert(0, str(phishing_agent_path))

print("=" * 80)
print("AGENT INITIALIZATION WITH LLM")
print("=" * 80)

# 1. VLLMのダミーAPIキーを設定（必須）
os.environ['OPENAI_API_KEY'] = 'dummy-key-for-vllm'
base_url = "http://192.168.100.71:30000"

# 2. LLM設定を強制有効化
print("\n[1] Fixing LLM Configuration")
print("-" * 40)

if 'external_data' not in globals():
    print("❌ external_data not found! Run Cell 2 first.")
else:
    if 'cfg' not in external_data:
        external_data['cfg'] = {}
    
    external_data['cfg']['llm'] = {
        'enabled': True,
        'provider': 'vllm',
        'base_url': base_url,
        'vllm_base_url': base_url,
        'model': 'Qwen/Qwen3-14B-FP8',
        'vllm_model': 'Qwen/Qwen3-14B-FP8'
    }
    print("✅ LLM configuration enabled")

# 3. phishpkgをクリア（古いものがあれば）
for key in list(sys.modules.keys()):
    if key.startswith('phishpkg'):
        del sys.modules[key]

# 4. LLMクライアントを作成
print("\n[2] Creating LLM Client")
print("-" * 40)

try:
    from langchain_community.llms import VLLMOpenAI
    
    llm_client = VLLMOpenAI(
        openai_api_base=base_url,
        openai_api_key="dummy",
        model_name="Qwen/Qwen3-14B-FP8",
        temperature=0.1,
        max_tokens=1024,
    )
    
    external_data['llm'] = llm_client
    print("✅ LLM client created and set")
    
except Exception as e:
    print(f"❌ LLM client creation failed: {e}")

# 5. Phase6配線（LLM有効）
print("\n[3] Wiring Phase6")
print("-" * 40)

try:
    from phishing_agent.phase6_wiring import wire_phase6
    wire_phase6(prefer_compat=True, fake_llm=False)  # fake_llm=Falseが重要！
    print("✅ Phase6 wired with real LLM")
except Exception as e:
    print(f"⚠️ Phase6 wiring: {e}")

# 6. LangGraphエージェントをインポート
print("\n[4] Importing LangGraphPhishingAgent")
print("-" * 40)

from phishing_agent.langgraph_module import LangGraphPhishingAgent
print("✅ LangGraphPhishingAgent imported")

# ========================================
# [COMMENTED OUT] Brand Detection Patching
# ========================================
# 改良版brand_impersonation_checkへの差し替えをコメントアウト
# 理由: Cell 3（改良版の定義）を削除したため、未定義エラーを回避
#
# print("\n[5] Patching Brand Detection")
# print("-" * 40)
#
# if 'phishpkg.tools_module' in sys.modules:
#     tools_module = sys.modules['phishpkg.tools_module']
#     
#     # 元の関数を保存
#     if hasattr(tools_module, 'brand_impersonation_check'):
#         tools_module._original_brand_check = tools_module.brand_impersonation_check
#     
#     # 改良版で置き換え
#     tools_module.brand_impersonation_check = brand_impersonation_check_enhanced
#     print("✅ Brand detection patched with enhanced version")
# else:
#     print("⚠️ phishpkg.tools_module not found, brand detection might not be enhanced")

# 8. エージェント初期化
print("\n[6] Initializing Agent")
print("-" * 40)

agent = LangGraphPhishingAgent(
    strict_mode=False,
    external_data=external_data
)
print("✅ Agent initialized")

# LLM設定の確認と強制有効化
if hasattr(agent, 'llm_config'):
    if not agent.llm_config.enabled:
        agent.llm_config.enabled = True
        print("✅ Forced LLM enabled=True")
    print(f"   LLM Status: enabled={agent.llm_config.enabled}")

# 9. 動作確認テスト
print("\n[7] Quick Verification Test")
print("-" * 40)

import time
test_domain = "test-amazon.com"
start = time.time()

try:
    result = agent.evaluate(test_domain, 0.35)
    elapsed = time.time() - start
    
    print(f"Test domain: {test_domain}")
    print(f"  Time: {elapsed:.2f}s")
    print(f"  is_phishing: {result.get('ai_is_phishing')}")
    
    if elapsed > 2.0:
        print("  ✅ LLM is working (slow response)")
    elif elapsed > 0.5:
        print("  ⚠️ LLM might be working")
    else:
        print("  ❌ LLM not working (too fast)")
        print("  ⚠️ Check VLLM server status")
    
    # Brand検出の確認
    brand_result = result.get('tool_results', {}).get('brand', {})
    if brand_result.get('data', {}).get('detected_issues'):
        brands = brand_result['data']['details'].get('detected_brands', [])
        print(f"  Brands detected: {brands}")
        print("  ✅ Brand detection is working")
    
except Exception as e:
    print(f"  ❌ Test failed: {e}")

print("\n" + "=" * 80)
print("AGENT READY FOR EVALUATION")
print("=" * 80)
print("✅ LLM: Enabled")
print("✅ Brand Detection: Using phishing_agent module version")
print("✅ Agent: Initialized")
print("\nProceed to Cell 5 for Random100 evaluation")
print("Expected time: ~8-10 minutes for 100 domains")
print("=" * 80)


AGENT INITIALIZATION WITH LLM

[1] Fixing LLM Configuration
----------------------------------------
✅ LLM configuration enabled

[2] Creating LLM Client
----------------------------------------
✅ LLM client created and set

[3] Wiring Phase6
----------------------------------------
✅ Phase6 wired with real LLM

[4] Importing LangGraphPhishingAgent
----------------------------------------
✅ LangGraphPhishingAgent imported

[6] Initializing Agent
----------------------------------------
✅ Agent initialized
   LLM Status: enabled=True

[7] Quick Verification Test
----------------------------------------
Test domain: test-amazon.com
  Time: 5.53s
  is_phishing: True
  ✅ LLM is working (slow response)

AGENT READY FOR EVALUATION
✅ LLM: Enabled
✅ Brand Detection: Using phishing_agent module version
✅ Agent: Initialized

Proceed to Cell 5 for Random100 evaluation
Expected time: ~8-10 minutes for 100 domains


In [7]:
# ============================================
# Cell 4.6: TLDリストの動的生成（データベース分析版）
# ※ DB接続必須。失敗時はエラー終了します。
# ============================================
import psycopg2
from psycopg2.extras import RealDictCursor
from collections import Counter
from urllib.parse import urlparse
import random
import numpy as np

print("🔧 Generating TLD data from Database Analysis...")

# 03_ai_agent_analysis.ipynb からの DB設定
DB_CONFIG = {
    'dbname': 'rapids_data',
    'user': 'postgres',
    'password': 'asomura',
    'host': 'localhost',
    'port': '5432'
}

# TLD抽出関数 (03からの移植)
def extract_tld(domain):
    """ドメインからTLDを抽出"""
    if not domain:
        return None
    # URLの場合はドメイン部分を抽出
    if '://' in domain:
        try:
            parsed = urlparse(domain)
            domain = parsed.netloc
        except:
            pass
    
    # ポート番号を削除
    domain = str(domain).split(':')[0]
    
    parts = domain.split('.')
    if len(parts) >= 2:
        # .co.jp, .ac.jp などの複合TLD対応
        if len(parts) >= 3 and parts[-2] in ['co', 'ac', 'or', 'ne', 'go']:
            return f'.{parts[-2]}.{parts[-1]}'
        else:
            return f'.{parts[-1]}'
    return None

try:
    # データベース接続テスト
    print(f"  🔌 Connecting to database: {DB_CONFIG['dbname']} at {DB_CONFIG['host']}...")
    conn = psycopg2.connect(**DB_CONFIG)
    cur = conn.cursor(cursor_factory=RealDictCursor)
    print("  ✅ Database connected successfully")

    # 1. フィッシングサイトの取得 (各ソースから)
    print("  📊 Analyzing phishing domains...")
    phishing_queries = [
        "SELECT cert_domain as domain FROM phishtank_entries WHERE cert_status = 'SUCCESS' AND cert_data IS NOT NULL",
        "SELECT domain FROM jpcert_phishing_urls WHERE status = 'SUCCESS' AND cert_data IS NOT NULL",
        "SELECT domain FROM certificates WHERE status = 'SUCCESS' AND cert_data IS NOT NULL"
    ]
    
    phishing_domains = []
    for query in phishing_queries:
        cur.execute(query)
        results = cur.fetchall()
        for row in results:
            if row['domain']: phishing_domains.append(row['domain'])
            
    if not phishing_domains:
        raise ValueError("No phishing domains found in database. Analysis cannot proceed.")

    # 2. 正常サイトの取得
    print("  📊 Analyzing trusted domains...")
    cur.execute("SELECT domain FROM trusted_certificates WHERE status = 'SUCCESS' AND cert_data IS NOT NULL")
    results = cur.fetchall()
    trusted_domains = [row['domain'] for row in results if row['domain']]
    
    conn.close()

    # 3. データバランス調整 (03と同様のロジック)
    # 重複除去
    phishing_domains_unique = list(set(phishing_domains))
    trusted_domains_unique = list(set(trusted_domains))
    
    print(f"    - Phishing unique: {len(phishing_domains_unique)}")
    print(f"    - Trusted unique: {len(trusted_domains_unique)}")
    
    # 少ない方に合わせてサンプリング
    random.seed(42)
    min_unique = min(len(phishing_domains_unique), len(trusted_domains_unique))
    
    if min_unique == 0:
        raise ValueError("Insufficient data for analysis (one of the datasets is empty).")
        
    phishing_balanced = random.sample(phishing_domains_unique, min(len(phishing_domains_unique), min_unique))
    trusted_balanced = random.sample(trusted_domains_unique, min(len(trusted_domains_unique), min_unique))

    # 4. TLD集計
    phishing_tlds = Counter([extract_tld(d) for d in phishing_balanced if extract_tld(d)])
    trusted_tlds = Counter([extract_tld(d) for d in trusted_balanced if extract_tld(d)])

    # 5. 危険度分析と分類
    dangerous_tlds = []
    
    for tld, phish_count in phishing_tlds.items():
        trust_count = trusted_tlds.get(tld, 0)
        # サンプル数が少なすぎるものは除外
        if phish_count >= 10:
            ratio = phish_count / (trust_count + 1)
            phish_pct = phish_count / len(phishing_balanced) * 100
            
            # 判定ロジック:
            # 1. 正常サイトで皆無(0件) かつ フィッシングで10件以上
            # 2. 比率が10倍以上 かつ フィッシング全体の0.1%以上
            if (trust_count == 0) or (ratio >= 10 and phish_pct >= 0.1):
                dangerous_tlds.append(tld)
    
    # ドットを除去して格納 (例: '.xyz' -> 'xyz')
    external_data['dangerous_tlds'] = [t.lstrip('.') for t in dangerous_tlds]

    # 正当なTLD
    legitimate_tlds = []
    for tld, count in trusted_tlds.most_common():
        if count >= 1000: # 統計的信頼性
            phish_count = phishing_tlds.get(tld, 0)
            ratio = phish_count / count
            if ratio < 0.5: # 正常サイトでの使用が2倍以上
                legitimate_tlds.append(tld)
    external_data['legitimate_tlds'] = [t.lstrip('.') for t in legitimate_tlds]

    # 中立的なTLD (主要TLDのうち、上記に入らなかったもの)
    neutral_candidates = ['.com', '.org', '.net', '.info', '.biz']
    neutral_tlds = []
    for tld in neutral_candidates:
        if tld not in dangerous_tlds and tld not in legitimate_tlds:
            neutral_tlds.append(tld)
    external_data['neutral_tlds'] = [t.lstrip('.') for t in neutral_tlds]
    
    # 統計情報を保存 (生のCounterオブジェクト)
    external_data['phishing_tld_stats'] = phishing_tlds

    print(f"  ✅ Generated {len(external_data['dangerous_tlds'])} dangerous TLDs")
    print(f"  ✅ Generated {len(external_data['legitimate_tlds'])} legitimate TLDs")

    # エージェント内の参照更新
    if 'agent' in globals():
        agent.external_data = external_data
        print("  ✅ Agent external_data updated")
        
    print("✅ TLD data patched via Database Analysis. Ready for evaluation.")

except Exception as e:
    print(f"\n❌ DATABASE ANALYSIS FAILED: {e}")
    print("⛔ Stopping execution to prevent inaccurate results using fallback data.")
    # 明示的にエラーを発生させて処理を止める
    raise RuntimeError("Database connection or analysis failed. Check DB_CONFIG and database status.") from eprint("✅ TLD data patched. Ready for evaluation.")

🔧 Generating TLD data from Database Analysis...
  🔌 Connecting to database: rapids_data at localhost...
  ✅ Database connected successfully
  📊 Analyzing phishing domains...
  📊 Analyzing trusted domains...
    - Phishing unique: 320409
    - Trusted unique: 450656
  ✅ Generated 28 dangerous TLDs
  ✅ Generated 26 legitimate TLDs
  ✅ Agent external_data updated
✅ TLD data patched via Database Analysis. Ready for evaluation.


In [8]:
# ============================================
# Cell 5: External Data内容確認（詳細版）
# ============================================

import json
from pprint import pprint

print("=" * 80)
print("EXTERNAL DATA INSPECTION (DETAILED)")
print("=" * 80)

# 0. External Dataの全体構造
print("\n[0] External Data Structure")
print("-" * 40)
print(f"Type: {type(external_data)}")
print(f"\nAll keys in external_data:")
for i, key in enumerate(sorted(external_data.keys()), 1):
    value = external_data[key]
    value_type = type(value).__name__
    
    # サイズ情報
    if isinstance(value, (list, dict)):
        size_info = f"(len={len(value)})"
    elif isinstance(value, str):
        size_info = f"(len={len(value)} chars)"
    else:
        size_info = ""
    
    print(f"  {i:2}. {key:30} {value_type:15} {size_info}")

# 1. Brand Keywords
print("\n" + "=" * 80)
print("[1] Brand Keywords")
print("-" * 40)
if 'brand_keywords' in external_data:
    brands = external_data['brand_keywords']
    print(f"Total count: {len(brands)}")
    print(f"Type: {type(brands)}")
    
    # 最初の10件
    print(f"\nFirst 10 brands:")
    for i, brand in enumerate(brands[:10], 1):
        print(f"  {i:2}. {brand}")
    
    # 最後の10件
    if len(brands) > 20:
        print(f"\n... ({len(brands) - 20} brands omitted) ...")
        print(f"\nLast 10 brands:")
        for i, brand in enumerate(brands[-10:], len(brands)-9):
            print(f"  {i:2}. {brand}")
    elif len(brands) > 10:
        print(f"\nRemaining brands:")
        for i, brand in enumerate(brands[10:], 11):
            print(f"  {i:2}. {brand}")
else:
    print("❌ brand_keywords not found")

# 2. Certificate Full Info Map (詳細版)
print("\n" + "=" * 80)
print("[2] Certificate Full Info Map (DETAILED)")
print("-" * 40)
if 'cert_full_info_map' in external_data:
    cert_map = external_data['cert_full_info_map']
    print(f"Total domains: {len(cert_map)}")
    print(f"Type: {type(cert_map)}")
    
    # サンプル表示（最大3件、全フィールド表示）
    sample_domains = list(cert_map.keys())[:3]
    print(f"\nDetailed sample (first {len(sample_domains)} domains):\n")
    
    for idx, domain in enumerate(sample_domains, 1):
        cert_info = cert_map[domain]
        print(f"[{idx}] Domain: {domain}")
        print(f"    Type: {type(cert_info)}")
        
        if isinstance(cert_info, dict):
            print(f"    All keys: {list(cert_info.keys())}")
            print(f"    Full content:")
            for key, value in cert_info.items():
                if isinstance(value, list) and len(value) > 3:
                    print(f"      {key}: {value[:3]} ... ({len(value)} total)")
                elif isinstance(value, dict):
                    print(f"      {key}: {value}")
                else:
                    print(f"      {key}: {value}")
        else:
            print(f"    Content: {cert_info}")
        print()
else:
    print("❌ cert_full_info_map not found")

# 3. Config内のTLD情報を探索
print("=" * 80)
print("[3] TLD Information Search")
print("-" * 40)

# 3-1. 直接のキーを確認
tld_keys = ['dangerous_tlds', 'legitimate_tlds', 'neutral_tlds', 
            'phishing_tld_stats', 'TLD_STATS', 'tld_stats']
print("\nSearching in external_data root:")
for key in tld_keys:
    if key in external_data:
        value = external_data[key]
        print(f"  ✅ {key}: {type(value).__name__} (len={len(value) if hasattr(value, '__len__') else 'N/A'})")
    else:
        print(f"  ❌ {key}: not found")

# 3-2. cfg内を探索
print("\nSearching in external_data['cfg']:")
if 'cfg' in external_data:
    cfg = external_data['cfg']
    print(f"  cfg type: {type(cfg)}")
    
    if isinstance(cfg, dict):
        print(f"  cfg keys: {list(cfg.keys())[:10]}...")
        
        # TLD関連をcfg内で探す
        for key in tld_keys:
            if key in cfg:
                value = cfg[key]
                print(f"  ✅ cfg['{key}']: {type(value).__name__} (len={len(value) if hasattr(value, '__len__') else 'N/A'})")
        
        # tld_analysis を探す
        if 'tld_analysis' in cfg:
            tld_analysis = cfg['tld_analysis']
            print(f"\n  Found 'tld_analysis' in cfg:")
            print(f"    Type: {type(tld_analysis)}")
            if isinstance(tld_analysis, dict):
                print(f"    Keys: {list(tld_analysis.keys())}")
else:
    print("  ❌ cfg not found")

# 4. High Risk Words の探索
print("\n" + "=" * 80)
print("[4] High Risk Words Search")
print("-" * 40)

hrw_keys = ['high_risk_words', 'HIGH_RISK_WORDS', 'high_risk']
print("Searching in external_data root:")
for key in hrw_keys:
    if key in external_data:
        value = external_data[key]
        print(f"  ✅ {key}: {type(value).__name__} (len={len(value) if hasattr(value, '__len__') else 'N/A'})")
        if isinstance(value, list) and len(value) > 0:
            print(f"     First 10: {value[:10]}")
    else:
        print(f"  ❌ {key}: not found")

# 5. Known Domains の探索
print("\n" + "=" * 80)
print("[5] Known Domains Search")
print("-" * 40)

kd_keys = ['known_domains', 'KNOWN_DOMAINS', 'legitimate_domains']
print("Searching in external_data root:")
for key in kd_keys:
    if key in external_data:
        value = external_data[key]
        print(f"  ✅ {key}: {type(value).__name__} (len={len(value) if hasattr(value, '__len__') else 'N/A'})")
        if isinstance(value, dict) and len(value) > 0:
            sample = dict(list(value.items())[:5])
            print(f"     Sample (5 entries):")
            for k, v in sample.items():
                print(f"       {k}: {v}")
    else:
        print(f"  ❌ {key}: not found")

# 6. 重要なデータが04-2にあるか確認のヒント
print("\n" + "=" * 80)
print("[6] Data Source Hint")
print("-" * 40)
print("Note: TLD lists and statistics might be in:")
print("  - 04-2_statistical_analysis.pkl")
print("  - config.json (tld_analysis section)")
print("  - Or need to be loaded separately")
print("\nCurrent pickle source: 04-3_llm_tools_setup_with_tools.pkl")
print("Consider loading 04-2 for complete TLD information.")

print("\n" + "=" * 80)
print("DETAILED INSPECTION COMPLETE")
print("=" * 80)

EXTERNAL DATA INSPECTION (DETAILED)

[0] External Data Structure
----------------------------------------
Type: <class 'dict'>

All keys in external_data:
   1. async_client                   dict            (len=0)
   2. brand_keywords                 list            (len=139)
   3. cert_full_info_map             dict            (len=4132)
   4. cfg                            dict            (len=4)
   5. dangerous_tlds                 list            (len=28)
   6. high_risk_words                list            (len=100)
   7. known_domains                  dict            (len=4132)
   8. legitimate_tlds                list            (len=26)
   9. llm                            VLLMOpenAI      
  10. neutral_tlds                   list            (len=3)
  11. phishing_tld_stats             Counter         (len=447)
  12. present_top                    list            (len=1)
  13. timestamp                      str             (len=26 chars)
  14. tools                          d

In [9]:
# ============================================
# Cell 4.5: ツール読み込み状態の最終確認
# ============================================
import sys
import os
import importlib

print("🔍 Tool Module Verification")
print("-" * 60)

# チェック対象
targets = [
    "phishing_agent.tools.certificate_analysis",
    "phishing_agent.tools.brand_impersonation_check"
]


for module_name in targets:
    short_name = module_name.split('.')[-1]
    print(f"\n📂 Checking: {short_name}...")
    
    try:
        # 1. キャッシュ消去（確実にファイルから読むため）
        if module_name in sys.modules:
            del sys.modules[module_name]
            
        # 2. インポート試行
        mod = importlib.import_module(module_name)
        
        # 3. パス確認
        file_path = mod.__file__
        
        if f"phishing_agent/tools/{short_name}.py" in file_path:
            print(f"  ✅ OK: New file loaded correctly")
            print(f"  📍 Path: {file_path}")
        else:
            print(f"  ⚠️ WARNING: Unexpected path -> {file_path}")
            
    except Exception as e:
        print(f"  ❌ ERROR: Failed to load -> {e}")

print("\n" + "-" * 60)
print("✅ 両方とも 'OK' なら、準備完了です。")
print("   'Kernel' -> 'Restart Kernel' を行い、最初から実行すれば")
print("   エージェントはこれらの新しいツールを使用します。")

🔍 Tool Module Verification
------------------------------------------------------------

📂 Checking: certificate_analysis...
  ✅ OK: New file loaded correctly
  📍 Path: /home/asomura/nextstep/phishing_agent/tools/certificate_analysis.py

📂 Checking: brand_impersonation_check...
  ✅ OK: New file loaded correctly
  📍 Path: /home/asomura/nextstep/phishing_agent/tools/brand_impersonation_check.py

------------------------------------------------------------
✅ 両方とも 'OK' なら、準備完了です。
   'Kernel' -> 'Restart Kernel' を行い、最初から実行すれば
   エージェントはこれらの新しいツールを使用します。


In [10]:
# ============================================
# Cell 5: Random100評価実行（全ツール詳細表示版）
# ============================================

import time
import pandas as pd
from datetime import datetime

print("[INFO] Starting FULL AGENT evaluation of 100 domains...")
print("=" * 80)

# LLM設定の確認
if 'agent' in globals() and hasattr(agent, 'llm_config'):
    llm_config = agent.llm_config
    if llm_config.enabled:
        print(f"[INFO] LLM initialized: {llm_config.model}")
    else:
        print("[WARNING] LLM is disabled - results may be limited")
else:
    print("[WARNING] Agent not properly initialized")

results = []
start_time = time.time()

# 表示用ヘルパー
def _fmt_issues(issues):
    return ", ".join(issues) if issues else "None"

# 各ドメインを評価
for idx, row in random100_df.iterrows():
    domain = row['domain']
    ml_prob = row['ml_probability']
    
    try:
        # --- エージェント実行 ---
        eval_start = time.time()
        result = agent.evaluate(domain, ml_prob)
        elapsed = time.time() - eval_start
        
        # --- 結果の抽出 ---
        # 1. 基本情報
        is_phishing = result.get('ai_is_phishing', False)
        confidence = result.get('ai_confidence', 0.0)
        risk_level = result.get('ai_risk_level', 'unknown')
        
        # 2. ツール結果 (tool_results) の取得
        # ※ LangGraphの構造上、graph_state または tool_results キーに入っている
        tool_res = {}
        if 'graph_state' in result and 'tool_results' in result['graph_state']:
            tool_res = result['graph_state']['tool_results']
        elif 'tool_results' in result:
            # 古いフォーマットの場合のフォールバック
            tool_res = result.get('tool_results', {})

        # 3. 各ツールの詳細抽出
        # [Brand]
        brand_res = tool_res.get('brand', {})
        brand_issues = brand_res.get('detected_issues', [])
        brand_details = brand_res.get('details', {})
        detected_brands = brand_details.get('detected_brands', [])
        
        # [Cert] - 今回追加したツール
        cert_res = tool_res.get('cert', {})
        cert_issues = cert_res.get('detected_issues', [])
        cert_score = cert_res.get('risk_score', 0.0)
        cert_details = cert_res.get('details', {})
        cert_issuer = cert_details.get('issuer', 'unknown')
        
        # [Domain]
        domain_res = tool_res.get('domain', {})
        domain_issues = domain_res.get('detected_issues', [])
        domain_details = domain_res.get('details', {})
        tld_cat = domain_details.get('tld_category', '-')
        
        # [Contextual]
        ctx_res = tool_res.get('contextual_risk_assessment', {})
        ctx_issues = ctx_res.get('detected_issues', [])
        
        # --- ログ表示 (リッチフォーマット) ---
        mark = "🔴" if is_phishing else "🟢"
        print(f"[{idx+1:3}/{len(random100_df)}] {mark} {domain:<35} (ML: {ml_prob:.3f} / Time: {elapsed:.2f}s)")
        
        # Brand info
        if brand_issues or detected_brands:
            print(f"    🏷️  Brand : {detected_brands} (Issues: {_fmt_issues(brand_issues)})")
        
        # Cert info (重要)
        if cert_issues or cert_score > 0:
            print(f"    🔒 Cert  : {_fmt_issues(cert_issues)} (Issuer: {cert_issuer})")
        
        # Domain info
        if domain_issues:
            print(f"    🌐 Domain: {_fmt_issues(domain_issues)} (TLD: {tld_cat})")
            
        # Contextual / Final
        print(f"    🏁 Final : Conf={confidence:.2f} ({risk_level})")
        print("-" * 60)

        # --- 保存用データの構築 ---
        results.append({
            'domain': domain,
            'ml_probability': ml_prob,
            'ai_is_phishing': is_phishing,
            'ai_confidence': confidence,
            'ai_risk_level': risk_level,
            'processing_time': elapsed,
            # Brand
            'brand_detected': bool(detected_brands),
            'brands': detected_brands,
            # Cert
            'cert_issues': cert_issues,
            'cert_issuer': cert_issuer,
            'cert_score': cert_score,
            # Domain
            'domain_issues': domain_issues,
            'tld_category': tld_cat,
            # Contextual
            'ctx_issues': ctx_issues,
            'error': None
        })
            
    except Exception as e:
        elapsed = time.time() - eval_start
        print(f"[{idx+1:3}/{len(random100_df)}] ❌ ERROR: {domain} - {str(e)}")
        results.append({
            'domain': domain,
            'ml_probability': ml_prob,
            'ai_is_phishing': False,
            'error': str(e)
        })

# --- 完了処理 ---
total_time = time.time() - start_time
print("=" * 80)
print(f"[INFO] Full evaluation complete! Total time: {total_time:.2f}s")

# DataFrame化と保存
results_df = pd.DataFrame(results)
timestamp = datetime.now().strftime("%Y-%m-%d_%H%M%S")
result_file = BASE_DIR / f"random100_full_eval_{timestamp}.csv"
results_df.to_csv(result_file, index=False)
print(f"[INFO] Detailed results saved to: {result_file}")

[INFO] Starting FULL AGENT evaluation of 100 domains...
[INFO] LLM initialized: Qwen/Qwen3-14B-FP8
[INFO] LLM initialized successfully: Qwen/Qwen3-14B-FP8 at http://192.168.100.71:30000/v1


/home/asomura/nextstep/phishing_agent/tools/brand_impersonation_check.py:263: LangChainDeprecationWarning: The method `BaseLLM.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = self.llm(prompt)


[  1/100] 🟢 jmbf.cn                             (ML: 0.454 / Time: 22.18s)
    🔒 Cert  : free_ca, no_org (Issuer: Let's Encrypt)
    🌐 Domain: short, dangerous_tld, random_pattern (TLD: dangerous)
    🏁 Final : Conf=0.45 (medium)
------------------------------------------------------------
[  2/100] 🔴 baidu-xiamen.cn                     (ML: 0.244 / Time: 6.13s)
    🏷️  Brand : ['x(compound)'] (Issues: brand_detected)
    🔒 Cert  : free_ca, no_org (Issuer: Let's Encrypt)
    🌐 Domain: dangerous_tld (TLD: dangerous)
    🏁 Final : Conf=0.70 (high)
------------------------------------------------------------
[  3/100] 🔴 tsjianye.cn                         (ML: 0.237 / Time: 4.34s)
    🔒 Cert  : free_ca, no_org (Issuer: Let's Encrypt)
    🌐 Domain: dangerous_tld (TLD: dangerous)
    🏁 Final : Conf=0.55 (medium-high)
------------------------------------------------------------
[  4/100] 🔴 internetku.id                       (ML: 0.130 / Time: 4.05s)
    🔒 Cert  : free_ca, no_org (Issuer: Se

In [11]:
# ============================================
# Cell 6: ツール別詳細統計サマリー
# ============================================
import numpy as np

# データの準備（エラーなしのデータのみ）
valid_df = results_df[results_df['error'].isna()].copy()

print("\n" + "=" * 80)
print("🔍 DETAILED TOOL ANALYSIS SUMMARY")
print("=" * 80)

# 1. 全体統計
phish_count = valid_df['ai_is_phishing'].sum()
total = len(valid_df)
print(f"\n📊 Overall Performance")
print(f"  Total Domains: {total}")
print(f"  Phishing Detected: {phish_count} ({phish_count/total*100:.1f}%)")
print(f"  Avg Processing Time: {valid_df['processing_time'].mean():.2f}s")

# 2. Certificate Analysis 統計 (今回注目の機能)
print(f"\n🔒 Certificate Analysis Stats")
# Issueごとのカウント
all_cert_issues = []
for issues in valid_df['cert_issues']:
    if isinstance(issues, list):
        all_cert_issues.extend(issues)
    elif isinstance(issues, str) and issues: # 文字列の場合のケア
        import ast
        try:
            all_cert_issues.extend(ast.literal_eval(issues))
        except:
            pass

if all_cert_issues:
    from collections import Counter
    cert_counts = Counter(all_cert_issues)
    for issue, count in cert_counts.most_common():
        print(f"  - {issue:20}: {count} domains")
else:
    print("  No certificate issues detected.")

# 発行者(Issuer)のTop5 (Free CAの確認など)
issuers = valid_df[valid_df['cert_issuer'] != 'unknown']['cert_issuer']
if not issuers.empty:
    print(f"  [Top Issuers]")
    print(issuers.value_counts().head(5).to_string(header=False))

# 3. Domain Analysis 統計
print(f"\n🌐 Domain Analysis Stats")
all_domain_issues = []
for issues in valid_df['domain_issues']:
    if isinstance(issues, list):
        all_domain_issues.extend(issues)

if all_domain_issues:
    dom_counts = Counter(all_domain_issues)
    for issue, count in dom_counts.most_common():
        print(f"  - {issue:20}: {count} domains")

# 4. Brand Analysis 統計
print(f"\n🏷️  Brand Analysis Stats")
brand_hits = valid_df[valid_df['brand_detected'] == True]
print(f"  Brand Detected: {len(brand_hits)} domains")
if not brand_hits.empty:
    all_brands = []
    for brands in brand_hits['brands']:
        if isinstance(brands, list): all_brands.extend(brands)
    print(f"  [Top Detected Brands]")
    print(pd.Series(all_brands).value_counts().head(5).to_string(header=False))

print("\n" + "=" * 80)
print(f"✅ Verification Complete. Check CSV for per-domain details.")


🔍 DETAILED TOOL ANALYSIS SUMMARY

📊 Overall Performance
  Total Domains: 100
  Phishing Detected: 82 (82.0%)
  Avg Processing Time: 6.75s

🔒 Certificate Analysis Stats
  - free_ca             : 99 domains
  - no_org              : 99 domains
  - wildcard            : 5 domains
  [Top Issuers]
Let's Encrypt            87
Google Trust Services     7
Sectigo Limited           3
Cloudflare, Inc.          1
DigiCert Inc              1

🌐 Domain Analysis Stats
  - dangerous_tld       : 84 domains
  - short               : 11 domains
  - random_pattern      : 2 domains

🏷️  Brand Analysis Stats
  Brand Detected: 19 domains
  [Top Detected Brands]
x(compound)        8
x(substring)       3
line(compound)     2
line(substring)    1
ntt(fuzzy)         1

✅ Verification Complete. Check CSV for per-domain details.


In [12]:
# セル1: 共通ヘルパ
from typing import List, Dict, Any
import json

def find_result(results: List[Dict[str, Any]], domain: str) -> Dict[str, Any]:
    """
    評価結果リストから特定ドメインの1件を返すヘルパ
    """
    target = domain.strip().lower()
    for r in results:
        d = str(r.get("domain", "")).strip().lower()
        if d == target:
            return r
    return {}


In [13]:
# セル2: 1ドメインの中身を全部見るデバッガ
from typing import Any, Dict

def debug_single_result(results: List[Dict[str, Any]], domain: str) -> None:
    res = find_result(results, domain)
    if not res:
        print(f"[WARN] domain '{domain}' が results に見つかりませんでした。")
        return
    
    gs: Dict[str, Any] = res.get("graph_state", {}) or {}
    tr: Dict[str, Any] = gs.get("tool_results", {}) or {}
    
    print("=" * 80)
    print(f"DEBUG for domain: {res.get('domain')}  (ML={res.get('ml_probability')})")
    print("-" * 80)
    print(f"ai_is_phishing: {res.get('ai_is_phishing')}  "
          f"ai_confidence: {res.get('ai_confidence')}  "
          f"ai_risk_level: {res.get('ai_risk_level')}")
    print()
    
    # --- precheck ---
    print(">>> PRECHECK HINTS")
    pre = gs.get("precheck_hints", {}) or {}
    print(json.dumps(pre, indent=2, ensure_ascii=False))
    
    # --- tools ---
    print("\n>>> TOOL RESULTS (brand / cert / domain / contextual_risk_assessment / contextual)")
    for key in ("brand", "cert", "domain", "contextual_risk_assessment", "contextual"):
        data = tr.get(key)
        if data is None:
            continue
        print(f"\n[{key}]")
        if isinstance(data, dict):
            print(f"  risk_score : {data.get('risk_score')}")
            print(f"  issues     : {data.get('detected_issues')}")
            details = data.get("details", {})
            if details:
                print("  details    :")
                print(json.dumps(details, indent=2, ensure_ascii=False))
            if data.get("_fallback"):
                print("  *** _fallback=True (tool 内でエラー or フォールバック) ***")
        else:
            print("  raw:", data)
    
    # --- final_assessment ---
    print("\n>>> FINAL ASSESSMENT (Phase4/6)")
    asmt = gs.get("final_assessment")
    if asmt is None:
        print("  <final_assessment がありません>")
    else:
        try:
            # Pydantic v2 BaseModel の場合
            asmt_dict = asmt.model_dump()
        except Exception:
            # すでに dict の場合
            asmt_dict = asmt if isinstance(asmt, dict) else {"_raw": str(asmt)}
        print(json.dumps(asmt_dict, indent=2, ensure_ascii=False))
    
    # --- fallback / trace ---
    print("\n>>> FALLBACK / TRACE")
    print("fallback_locations:", gs.get("fallback_locations"))
    if "decision_trace" in gs:
        print("decision_trace:")
        print(json.dumps(gs["decision_trace"], indent=2, ensure_ascii=False))
    else:
        print("decision_trace: <none>")
    
    print("=" * 80)


In [14]:
# セル3: Phase6 ポリシーに渡る入力の要約
from phishing_agent.llm_final_decision import _summarize_tool_signals  # type: ignore

def debug_policy_inputs(results: List[Dict[str, Any]], domain: str) -> None:
    res = find_result(results, domain)
    if not res:
        print(f"[WARN] domain '{domain}' が results に見つかりませんでした。")
        return
    
    gs: Dict[str, Any] = res.get("graph_state", {}) or {}
    tr: Dict[str, Any] = gs.get("tool_results", {}) or {}
    pre: Dict[str, Any] = gs.get("precheck_hints", {}) or {}
    ml = float(res.get("ml_probability", 0.0) or 0.0)
    
    tsum = _summarize_tool_signals(tr)
    
    print("=" * 80)
    print(f"POLICY INPUTS for domain: {res.get('domain')}  (ML={ml:.3f})")
    print("-" * 80)
    print("precheck:")
    print(f"  tld_category          : {pre.get('tld_category')}")
    print(f"  domain_length_category: {pre.get('domain_length_category')}")
    print(f"  quick_risk            : {pre.get('quick_risk')}")
    print()
    
    print("brand:")
    print(f"  risk_score : {tsum['brand']['risk_score']}")
    print(f"  issues     : {tsum['brand']['issues']}")
    print(f"  brands     : {tsum['brand']['brands']}")
    print()
    
    print("cert:")
    print(f"  risk_score : {tsum['cert']['risk_score']}")
    print(f"  issues     : {tsum['cert']['issues']}")
    print(f"  details    : {tsum['cert']['details']}")
    print()
    
    print("domain:")
    print(f"  risk_score : {tsum['domain']['risk_score']}")
    print(f"  issues     : {tsum['domain']['issues']}")
    print(f"  details    : {tsum['domain']['details']}")
    print()
    
    print("contextual (used by Phase6):")
    print(f"  risk_score : {tsum['contextual']['risk_score']}")
    print(f"  issues     : {tsum['contextual']['issues']}")
    print()
    
    print(f"baseline_risk (max of brand/cert/domain/contextual): {tsum['baseline_risk']}")
    print("=" * 80)


In [15]:
# セル4: R1 候補一覧（ml<0.2 かつ free_ca+no_org）
def list_r1_candidates(results: List[Dict[str, Any]], ml_th: float = 0.2, limit: int = 20) -> None:
    rows = []
    for r in results:
        gs = r.get("graph_state", {}) or {}
        tr = gs.get("tool_results", {}) or {}
        cert = tr.get("cert", {}) or {}
        issues = set(cert.get("detected_issues", []) or [])
        ml = float(r.get("ml_probability", 0.0) or 0.0)
        
        if ml < ml_th and {"free_ca", "no_org"} <= issues:
            ctx = tr.get("contextual_risk_assessment") or tr.get("contextual") or {}
            rows.append({
                "domain": r.get("domain"),
                "ml": ml,
                "cert_issues": sorted(list(issues)),
                "ctx_score": float(ctx.get("risk_score", 0.0) or 0.0),
                "ctx_issues": ctx.get("detected_issues", []),
                "ai_is_phishing": r.get("ai_is_phishing"),
                "ai_confidence": r.get("ai_confidence"),
            })
    
    rows = sorted(rows, key=lambda x: x["ml"])
    print(f"R1候補 (ml<{ml_th}, free_ca+no_org) 件数: {len(rows)}")
    print("-" * 80)
    for row in rows[:limit]:
        print(
            f"{row['domain']:<30s} "
            f"ml={row['ml']:.3f}  "
            f"cert={row['cert_issues']}  "
            f"ctx={row['ctx_score']:.3f}  "
            f"final={row['ai_is_phishing']} (conf={row['ai_confidence']:.2f})"
        )
    print("-" * 80)


In [16]:
# 1) R1候補をざっと確認
list_r1_candidates(results)

# 2) その中から、例えば "internetku.id" など
debug_single_result(results, "internetku.id")
debug_policy_inputs(results, "internetku.id")


R1候補 (ml<0.2, free_ca+no_org) 件数: 0
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
DEBUG for domain: internetku.id  (ML=0.12955579)
--------------------------------------------------------------------------------
ai_is_phishing: True  ai_confidence: 0.55  ai_risk_level: medium-high

>>> PRECHECK HINTS
{}

>>> TOOL RESULTS (brand / cert / domain / contextual_risk_assessment / contextual)

>>> FINAL ASSESSMENT (Phase4/6)
  <final_assessment がありません>

>>> FALLBACK / TRACE
fallback_locations: None
decision_trace: <none>
POLICY INPUTS for domain: internetku.id  (ML=0.130)
--------------------------------------------------------------------------------
precheck:
  tld_category          : None
  domain_length_category: None
  quick_risk            : None

brand:
  risk_score : 0.0
  issues     : []
  brands     : []

cert:
  risk_score : 0.0
  issues     : []
  details    : {}

do

In [17]:
# エラー詳細を見るデバッガ
def debug_error_details(results, domain: str) -> None:
    res = find_result(results, domain)
    if not res:
        print(f"[WARN] domain '{domain}' が results に見つかりませんでした。")
        return
    
    print("=" * 80)
    print(f"ERROR DETAILS for domain: {res.get('domain')}  (ML={res.get('ml_probability')})")
    print("-" * 80)
    
    # トップレベルのキー一覧
    print("[keys]")
    print(sorted(res.keys()))
    print()
    
    # 成功/失敗フラグとエラー情報
    print("[status]")
    print("  success       :", res.get("success"))
    print("  phase         :", res.get("phase"))
    print("  error_category:", res.get("error_category"))
    print("  error         :", res.get("error"))
    print()
    
    # トレースバック（長い場合は先頭だけ）
    tb = res.get("traceback")
    if tb:
        print("[traceback]")
        # 必要なら先頭だけにしてもOK
        print(tb)
    else:
        print("[traceback]")
        print("  <no traceback field>")
    
    print("=" * 80)


# 全体のエラー件数もざっと見る
def summarize_errors(results):
    total = len(results)
    errs = [r for r in results if r.get("success") is False]
    print(f"total={total}, errors={len(errs)}")
    if errs:
        print("  sample error domains:", [e.get("domain") for e in errs[:5]])


In [18]:
# 全体でどれくらいエラーになっているか
summarize_errors(results)

# 問題の internetku.id のエラー詳細
debug_error_details(results, "internetku.id")


total=100, errors=0
ERROR DETAILS for domain: internetku.id  (ML=0.12955579)
--------------------------------------------------------------------------------
[keys]
['ai_confidence', 'ai_is_phishing', 'ai_risk_level', 'brand_detected', 'brands', 'cert_issuer', 'cert_issues', 'cert_score', 'ctx_issues', 'domain', 'domain_issues', 'error', 'ml_probability', 'processing_time', 'tld_category']

[status]
  success       : None
  phase         : None
  error_category: None
  error         : None

[traceback]
  <no traceback field>


In [19]:
# === Cell: agent.evaluate の中身を見るデバッガ ===
from typing import Any, Dict
import json

def debug_agent_single(domain: str, ml_probability: float) -> Dict[str, Any]:
    """
    LangGraphPhishingAgent.evaluate(domain, ml_probability) を1件実行して、
    - precheck_hints
    - tool_results (brand/cert/domain/contextual)
    - final_assessment
    を全部プリントする。
    """
    print("=" * 80)
    print(f"[DEBUG] agent.evaluate for {domain} (ML={ml_probability:.3f})")
    print("=" * 80)

    # 1. エージェント実行（元の agent をそのまま使う）
    res: Dict[str, Any] = agent.evaluate(domain, ml_probability)
    print("\n[TOP-LEVEL KEYS]")
    print(sorted(res.keys()))

    # graph_state の存在確認
    gs: Dict[str, Any] = res.get("graph_state", {}) or {}
    print("\n[GRAPH_STATE KEYS]")
    print(sorted(gs.keys()))

    # 2. precheck_hints
    print("\n>>> PRECHECK HINTS")
    pre = gs.get("precheck_hints", {}) or {}
    print(json.dumps(pre, indent=2, ensure_ascii=False))

    # 3. tool_results の各ツール
    print("\n>>> TOOL RESULTS")
    tr: Dict[str, Any] = gs.get("tool_results", {}) or {}
    print("tool keys:", list(tr.keys()))

    for key in ("brand", "cert", "domain", "contextual_risk_assessment", "contextual"):
        data = tr.get(key)
        if data is None:
            continue
        print(f"\n[{key}]")
        if isinstance(data, dict):
            print(f"  risk_score : {data.get('risk_score')}")
            print(f"  issues     : {data.get('detected_issues')}")
            details = data.get("details", {})
            if details:
                print("  details    :")
                print(json.dumps(details, indent=2, ensure_ascii=False))
            if data.get("_fallback"):
                print("  *** _fallback=True (tool 内でエラー or フォールバック) ***")
        else:
            print("  raw:", data)

    # 4. final_assessment（Phase6 + LLM の最終判定）
    print("\n>>> FINAL ASSESSMENT")
    fa = gs.get("final_assessment")
    if fa is None:
        print("  <None>")
    else:
        try:
            # Pydantic v2 BaseModel の場合
            fa_dict = fa.model_dump()
        except Exception:
            fa_dict = fa if isinstance(fa, dict) else {"_raw": str(fa)}
        print(json.dumps(fa_dict, indent=2, ensure_ascii=False))

    # 5. decision_trace (R1/R2/R3など)
    if "decision_trace" in gs:
        print("\n>>> DECISION TRACE")
        print(json.dumps(gs["decision_trace"], indent=2, ensure_ascii=False))
    else:
        print("\n>>> DECISION TRACE")
        print("  <none>")

    print("=" * 80)
    return res


In [20]:
# === Cell: 実行例 ===

# さっきのログに合わせて ML 値を入れる
debug_result = debug_agent_single("internetku.id", 0.12955579)


[DEBUG] agent.evaluate for internetku.id (ML=0.130)

[TOP-LEVEL KEYS]
['ai_confidence', 'ai_is_phishing', 'ai_risk_level', 'detected_brands', 'domain', 'graph_state', 'llm_driven', 'ml_probability', 'module_version', 'phase', 'processing_time', 'reasoning', 'success', 'tools_used']

[GRAPH_STATE KEYS]
['current_step', 'domain', 'error', 'fallback_count', 'fallback_locations', 'final_assessment', 'ml_probability', 'precheck_hints', 'retry_count', 'selected_tools', 'tool_results']

>>> PRECHECK HINTS
{
  "ml_category": "very_low",
  "ml_paradox": false,
  "tld_category": "unknown",
  "brand_detected": false,
  "potential_brands": [],
  "domain_length_category": "normal",
  "quick_risk": 0.05,
  "recommended_tools": [
    "brand_impersonation_check",
    "certificate_analysis",
    "short_domain_analysis"
  ],
  "etld1": {
    "registered_domain": "internetku.id",
    "domain": "internetku",
    "suffix": "id",
    "subdomain": ""
  },
  "stats": {
    "phishing_tld_weight": 0.0,
    "hig

In [21]:
debug_agent_single("internetku.id", 0.12955579)


[DEBUG] agent.evaluate for internetku.id (ML=0.130)

[TOP-LEVEL KEYS]
['ai_confidence', 'ai_is_phishing', 'ai_risk_level', 'detected_brands', 'domain', 'graph_state', 'llm_driven', 'ml_probability', 'module_version', 'phase', 'processing_time', 'reasoning', 'success', 'tools_used']

[GRAPH_STATE KEYS]
['current_step', 'domain', 'error', 'fallback_count', 'fallback_locations', 'final_assessment', 'ml_probability', 'precheck_hints', 'retry_count', 'selected_tools', 'tool_results']

>>> PRECHECK HINTS
{
  "ml_category": "very_low",
  "ml_paradox": false,
  "tld_category": "unknown",
  "brand_detected": false,
  "potential_brands": [],
  "domain_length_category": "normal",
  "quick_risk": 0.05,
  "recommended_tools": [
    "brand_impersonation_check",
    "certificate_analysis",
    "short_domain_analysis"
  ],
  "etld1": {
    "registered_domain": "internetku.id",
    "domain": "internetku",
    "suffix": "id",
    "subdomain": ""
  },
  "stats": {
    "phishing_tld_weight": 0.0,
    "hig

{'domain': 'internetku.id',
 'ml_probability': 0.12955579,
 'ai_is_phishing': True,
 'ai_confidence': 0.55,
 'ai_risk_level': 'medium-high',
 'detected_brands': [],
 'reasoning': 'The domain internetku.id has a low risk level based on the analysis. The contextual risk score is 0.415, which is below the threshold of 0.5, and no significant risk factors were identified.',
 'success': True,
 'processing_time': 4.836447942070663,
 'tools_used': ['brand', 'cert', 'domain', 'contextual_risk_assessment'],
 'llm_driven': True,
 'phase': 'phase4_v1.3',
 'graph_state': {'domain': 'internetku.id',
  'ml_probability': 0.12955579,
  'current_step': 'completed',
  'precheck_hints': {'ml_category': 'very_low',
   'ml_paradox': False,
   'tld_category': 'unknown',
   'brand_detected': False,
   'potential_brands': [],
   'domain_length_category': 'normal',
   'quick_risk': 0.05,
   'recommended_tools': ['brand_impersonation_check',
    'certificate_analysis',
    'short_domain_analysis'],
   'etld1': 

In [22]:
# ============================================
# Cell FULL-1: FULL評価用 eval_df を 04-2 / 04-3 の pickle から構築
# ============================================
import pickle
from urllib.parse import urlparse
import pandas as pd
from pathlib import Path

print("\n" + "=" * 80)
print("[FULL] Building eval_df from artifacts (04-2 / 04-3)")
print("=" * 80)

# 候補となるpickleファイル（必要に応じてパスを追加）
PICKLE_CANDIDATES = [
    HANDOFF_DIR / "04-2_statistical_analysis.pkl",
    HANDOFF_DIR / "04-3_random_eval_context.pkl",
    Path("/home/asomura/nextstep/artifacts/20251030_004055/handoff/04-2_statistical_analysis.pkl"),
    Path("/home/asomura/nextstep/artifacts/20251030_004055/handoff/04-3_random_eval_context.pkl"),
]

def _collect_dfs(obj, acc):
    """任意のオブジェクトから再帰的に DataFrame を収集"""
    if obj is None:
        return
    if isinstance(obj, pd.DataFrame):
        acc.append(obj)
        return
    if isinstance(obj, dict):
        for v in obj.values():
            _collect_dfs(v, acc)
    elif isinstance(obj, (list, tuple)):
        for v in obj:
            _collect_dfs(v, acc)

def _score_candidate_df(df: pd.DataFrame) -> int:
    """
    評価用DFをスコアリングする簡易関数。
    - domain/fqdn/hostname/url 系の列があれば +2
    - ml_probability/probability/score/proba 系の列があれば +2
    - (tld,count)だけの統計テーブルは強く減点
    - 行数が多い (>= 500) ものを強く優先
    - 100件前後のサンプルDFは明示的に減点
    """
    cols = [c.lower() for c in df.columns]
    has_domain = any(any(k in c for k in ["domain", "fqdn", "hostname", "host", "requested_host", "url"]) for c in cols)
    has_ml = any(any(k in c for k in ["ml_probability", "ml_prob", "probability", "prediction_proba", "score", "pred_proba", "proba", "confidence"]) for c in cols)
    score = 0
    if has_domain:
        score += 2
    if has_ml:
        score += 2
    # (tld, count) だけの統計テーブルは明示的に除外したいので減点
    if set(cols) == {"tld", "count"}:
        score -= 5
    # 行数による重み付け: 本番用の大きなDFを優先
    n_rows = len(df)
    if n_rows >= 500:
        score += 3  # 明確にフルセットを優先
    elif n_rows >= 200:
        score += 1
    # 100件前後のサンプル（Random100相当）は候補として弱い
    if 80 <= n_rows <= 150:
        score -= 2
    # 行数が極端に少ないものはさらに減点
    if n_rows < 50:
        score -= 2
    return score
def _normalize_eval_df_full(df: pd.DataFrame) -> pd.DataFrame:
    """列名の揺れに対応して domain, ml_probability の2列に正規化 (FULL用)"""
    print("[DEBUG][FULL] columns:", list(df.columns))

    # lower → 元名のマップ
    lower2orig = {c.lower(): c for c in df.columns}

    # 1) domain候補
    domain_candidates = ["domain", "fqdn", "domain_name", "hostname", "host", "requested_host", "url"]
    domain_key = None
    for key in domain_candidates:
        if key in lower2orig:
            domain_key = lower2orig[key]
            break
    if domain_key is None:
        # 部分一致で fallback
        for c in df.columns:
            lc = c.lower()
            if any(kw in lc for kw in ["domain", "fqdn", "host", "url"]):
                domain_key = c
                print(f"[DEBUG][FULL] domain fallback: {c}")
                break

    # 2) ml_probability候補
    ml_candidates = [
        "ml_probability", "ml_prob", "probability", "prediction_proba",
        "score", "pred_proba", "proba", "confidence"
    ]
    mlp_key = None
    for key in ml_candidates:
        if key in lower2orig:
            mlp_key = lower2orig[key]
            break
    if mlp_key is None:
        float_cols = [c for c in df.columns if pd.api.types.is_float_dtype(df[c])]
        for c in float_cols:
            lc = c.lower()
            if any(kw in lc for kw in ["prob", "score", "pred"]):
                mlp_key = c
                print(f"[DEBUG][FULL] ml_probability fallback: {c}")
                break

    if domain_key is None or mlp_key is None:
        print("[ERROR][FULL] domain_key or mlp_key not found")
        return None

    tmp = df[[domain_key, mlp_key]].copy()
    tmp.columns = ["domain", "ml_probability"]

    # URL形式が混ざっていたらドメイン部分だけ抽出
    if tmp["domain"].astype(str).str.contains("://").any():
        def _to_domain(x: str) -> str:
            if not isinstance(x, str):
                return ""
            if "://" in x:
                netloc = urlparse(x).netloc
            else:
                netloc = x
            # user@host:port → host
            netloc = netloc.split("@")[-1].split(":")[0]
            return netloc.lower()
        tmp["domain"] = tmp["domain"].map(_to_domain)

    # クリーンアップ
    tmp["domain"] = tmp["domain"].astype(str).str.strip().str.lower()
    tmp = tmp[tmp["domain"].str.len() > 0]

    tmp["ml_probability"] = pd.to_numeric(tmp["ml_probability"], errors="coerce")
    tmp = tmp[(tmp["ml_probability"] >= 0.0) & (tmp["ml_probability"] <= 1.0)]
    tmp = tmp.dropna(subset=["ml_probability"]).reset_index(drop=True)

    return tmp

# === 実際に pickle から DataFrame を収集 ===
all_dfs = []
used_paths = []
for p in PICKLE_CANDIDATES:
    try:
        if not p.exists():
            continue
        print(f"[FULL] Loading pickle: {p}")
        with open(p, "rb") as f:
            obj = pickle.load(f)
        _collect_dfs(obj, all_dfs)
        used_paths.append(str(p))
    except Exception as e:
        print(f"[WARN][FULL] Failed to load {p}: {e}")

if not all_dfs:
    raise RuntimeError("04-2 / 04-3 のpickleから DataFrame を収集できませんでした。")

print(f"[FULL] Collected {len(all_dfs)} DataFrame candidates from:")
for p in used_paths:
    print(f"  - {p}")

# 各DFをスコアリングして最も評価用らしいものを選択
scored = [(i, _score_candidate_df(df), df) for i, df in enumerate(all_dfs)]
scored.sort(key=lambda x: x[1], reverse=True)
best_idx, best_score, best_df = scored[0]

print(f"[FULL] Selected candidate idx={best_idx} with score={best_score}")
print(f"[FULL] Selected columns: {list(best_df.columns)}")
print(f"[FULL] Selected rows: {len(best_df)}")

if best_score <= 0:
    raise RuntimeError("評価用に適した DataFrame が見つかりません (score<=0)。")

# 正規化して eval_df として利用
norm_df = _normalize_eval_df_full(best_df)
if norm_df is None:
    raise RuntimeError("[FULL] DataFrameの正規化に失敗しました")

print(f"[INFO][FULL] Source rows (normalized, FULL): {len(norm_df)}")

eval_df = norm_df.reset_index(drop=True)
print(f"[INFO][FULL] Using FULL dataset for evaluation: {len(eval_df)} domains")

# ログとしてCSV保存
LOGS_DIR.mkdir(parents=True, exist_ok=True)
out_csv = LOGS_DIR / "eval_domains_full_latest.csv"
eval_df.to_csv(out_csv, index=False, encoding="utf-8")
print(f"[INFO][FULL] Full eval list saved to: {out_csv}")

# ML確率の分布を確認
print("\n[FULL][ML Probability Distribution]")
print(f"  Min:    {eval_df['ml_probability'].min():.3f}")
print(f"  Max:    {eval_df['ml_probability'].max():.3f}")
print(f"  Mean:   {eval_df['ml_probability'].mean():.3f}")
print(f"  Median: {eval_df['ml_probability'].median():.3f}")

high_risk_full = eval_df[eval_df['ml_probability'] > 0.4]
print(f"  High risk (>0.4): {len(high_risk_full)} domains")



[FULL] Building eval_df from artifacts (04-2 / 04-3)
[FULL] Loading pickle: /home/asomura/nextstep/artifacts/20251030_004055/handoff/04-2_statistical_analysis.pkl
[FULL] Collected 4 DataFrame candidates from:
  - /home/asomura/nextstep/artifacts/20251030_004055/handoff/04-2_statistical_analysis.pkl
[FULL] Selected candidate idx=0 with score=7
[FULL] Selected columns: ['domain', 'source', 'prediction_proba', 'confidence']
[FULL] Selected rows: 4132
[DEBUG][FULL] columns: ['domain', 'source', 'prediction_proba', 'confidence']
[INFO][FULL] Source rows (normalized, FULL): 4132
[INFO][FULL] Using FULL dataset for evaluation: 4132 domains
[INFO][FULL] Full eval list saved to: /home/asomura/nextstep/artifacts/2025-11-22_020618/logs/eval_domains_full_latest.csv

[FULL][ML Probability Distribution]
  Min:    0.001
  Max:    0.500
  Mean:   0.209
  Median: 0.176
  High risk (>0.4): 576 domains


In [ ]:
# ============================================
# Cell FULL-2: FULL AGENT evaluation for ALL domains in eval_df
# ============================================

import time
import pandas as pd
from datetime import datetime

if 'eval_df' not in globals():
    raise RuntimeError("eval_df が定義されていません。先に Cell FULL-1 を実行してください。")

print(f"[INFO] Starting FULL AGENT evaluation of {len(eval_df)} domains...")
print("=" * 80)

# LLM設定の確認
if 'agent' in globals() and hasattr(agent, 'llm_config'):
    llm_config = agent.llm_config
    if llm_config.enabled:
        print(f"[INFO] LLM initialized: {llm_config.model}")
    else:
        print("[WARNING] LLM is disabled - results may be limited")
else:
    print("[WARNING] Agent not properly initialized")

results = []
start_time = time.time()

def _fmt_issues(issues):
    return ", ".join(issues) if issues else "None"

for idx, row in eval_df.iterrows():
    domain = row['domain']
    ml_prob = row['ml_probability']

    try:
        eval_start = time.time()
        result = agent.evaluate(domain, ml_prob)
        elapsed = time.time() - eval_start

        is_phishing = result.get('ai_is_phishing', False)
        confidence = result.get('ai_confidence', 0.0)
        risk_level = result.get('ai_risk_level', 'unknown')

        tool_res = {}
        if 'graph_state' in result and 'tool_results' in result['graph_state']:
            tool_res = result['graph_state']['tool_results']
        elif 'tool_results' in result:
            tool_res = result.get('tool_results', {})

        brand_res = tool_res.get('brand', {})
        brand_issues = brand_res.get('detected_issues', [])
        brand_details = brand_res.get('details', {})
        detected_brands = brand_details.get('detected_brands', [])

        cert_res = tool_res.get('cert', {})
        cert_issues = cert_res.get('detected_issues', [])
        cert_score = cert_res.get('risk_score', 0.0)
        cert_details = cert_res.get('details', {})
        cert_issuer = cert_details.get('issuer', 'unknown')

        domain_res = tool_res.get('domain', {})
        domain_issues = domain_res.get('detected_issues', [])
        domain_details = domain_res.get('details', {})
        tld_cat = domain_details.get('tld_category', '-')

        ctx_res = tool_res.get('contextual_risk_assessment', {})
        ctx_issues = ctx_res.get('detected_issues', [])

        mark = "🔴" if is_phishing else "🟢"
        print(f"[{idx+1:5}/{len(eval_df):5}] {mark} {domain:<35} (ML: {ml_prob:.3f} / Time: {elapsed:.2f}s)")
        print(f"    🏁 Final : Conf={confidence:.2f} ({risk_level})")
        if detected_brands:
            print(f"    🏷️  Brand : {detected_brands} (Issues: {_fmt_issues(brand_issues)})")
        if cert_issues:
            print(f"    🔒 Cert  : {_fmt_issues(cert_issues)} (Issuer: {cert_issuer})")
        if domain_issues:
            print(f"    🌐 Domain: {_fmt_issues(domain_issues)} (TLD: {tld_cat})")
        if ctx_issues:
            print(f"    📌 Ctx   : {_fmt_issues(ctx_issues)}")
        print("-" * 60)

        results.append({
            'domain': domain,
            'ml_probability': ml_prob,
            'ai_is_phishing': is_phishing,
            'ai_confidence': confidence,
            'ai_risk_level': risk_level,
            'processing_time': elapsed,
            'brand_detected': bool(detected_brands),
            'brands': detected_brands,
            'cert_issues': cert_issues,
            'cert_issuer': cert_issuer,
            'cert_score': cert_score,
            'domain_issues': domain_issues,
            'tld_category': tld_cat,
            'ctx_issues': ctx_issues,
            'error': None,
        })

    except Exception as e:
        print(f"[{idx+1:5}/{len(eval_df):5}] ❌ ERROR: {domain} - {str(e)}")
        results.append({
            'domain': domain,
            'ml_probability': ml_prob,
            'ai_is_phishing': False,
            'ai_confidence': 0.0,
            'ai_risk_level': 'error',
            'processing_time': None,
            'brand_detected': False,
            'brands': [],
            'cert_issues': [],
            'cert_issuer': 'unknown',
            'cert_score': 0.0,
            'domain_issues': [],
            'tld_category': '-',
            'ctx_issues': [],
            'error': str(e),
        })

total_time = time.time() - start_time
print("=" * 80)
print(f"[INFO] Full evaluation complete! Total time: {total_time:.2f}s")

results_df = pd.DataFrame(results)
timestamp = datetime.now().strftime("%Y-%m-%d_%H%M%S")
result_file = BASE_DIR / f"full_eval_{timestamp}.csv"
results_df.to_csv(result_file, index=False)
print(f"[INFO] Detailed FULL results saved to: {result_file}")


[INFO] Starting FULL AGENT evaluation of 4132 domains...
[INFO] LLM initialized: Qwen/Qwen3-14B-FP8
[    1/ 4132] 🟢 winrneteik.icu                      (ML: 0.324 / Time: 5.21s)
    🏁 Final : Conf=0.29 (low)
    🔒 Cert  : free_ca, no_org (Issuer: Let's Encrypt)
    🌐 Domain: dangerous_tld (TLD: dangerous)
    📌 Ctx   : multiple_risk_factors, known_domain
------------------------------------------------------------
[    2/ 4132] 🔴 mecano-normand.com                  (ML: 0.070 / Time: 3.96s)
    🏁 Final : Conf=0.55 (medium-high)
    🔒 Cert  : free_ca, no_org (Issuer: Let's Encrypt)
    🌐 Domain: dangerous_tld (TLD: dangerous)
    📌 Ctx   : ml_paradox, multiple_risk_factors, known_domain
------------------------------------------------------------
